In [1]:
#Percy's HOBO processing code, with small modifications made by Eren 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,scatter,savefig,figure,colorbar,suptitle,close,show,subplot,rgrids
import pylab
from matplotlib.mlab import csv2rec
import matplotlib as mpl
import datetime
from matplotlib.dates import DateFormatter
from time import mktime
from scipy.interpolate import interp1d
import pickle

In [2]:
# calibration relationships
calnew = csv2rec('/Users/Eren/Desktop/data/HOBO/linregressions.csv')
calK = csv2rec('/Users/Eren/Desktop/data/HOBO/calibrations_Kevinsensors.csv')
#These are record arrays. Each row contains the regression coefficients for a particular sensor. The entries in the rows
#are sensor#, mT(i.e. the slope coeffient for temperature), bT (i.e the intercept coefficient for temperature), mRH, and
#finally bRH. This is documented in the dtype row which shows up at the bottom.
#Below is how you look at the header of a record array:
calnew[0:10]

rec.array([(32, 0.953976046759, 0.64074421949, 1.01009562711, 0.820251646579),
 (33, 0.93028776209, 1.20901119849, 0.984301625197, 1.40979323766),
 (34, 0.990206058501, 0.36112728793, 0.965737323224, 0.663595892954),
 (35, 0.988577614232, 0.485784725522, 0.95853246459, 0.863696131504),
 (36, 1.03663052043, -0.547961711482, 1.00619079225, -0.333523569232),
 (37, 1.03843884483, -0.83563849316, 1.01428928775, -0.362569647039),
 (38, 1.03702566634, -0.796974732731, 1.01254902482, -0.381930208069),
 (39, 1.0567588972, -1.14557548562, 1.0164639295, -0.502797080886),
 (40, 1.06455479323, -1.36369645284, 1.05161790422, -1.58042902666),
 (41, 1.03886092231, -0.978935812207, 0.992379717168, 0.0548943797267)], 
          dtype=[('sensor', '<i8'), ('mt', '<f8'), ('bt', '<f8'), ('mrh', '<f8'), ('brh', '<f8')])

In [3]:
#To select a specific value, use bracket notation twice as follows:
calnew[0][0]

32

In [4]:
#In this section, we are creating a dictionary object that contains the file path tails relevant to each sensor number.
#In the loop that uses these file path tails, there is a place to specify the file path head which is common to each 
#but unique to your computer, so look out for that later. To get this to run on your computer you need to change it.

#Note also that you can relate sensor number to position on the hill by looking in both HoboDeviceProperties.xlsx and
#SensorCartoonMap.pptx
files = dict()
# 795 = 1-3
files['795'] = ['/25 August 2010/station1-3.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373795.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373795_0.csv','/6 Feb 2012/2373795.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373795.csv',\
    '/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373795_0.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373795.csv']
# 796 = 2-4
files['796'] = ['/25 August 2010/station2-4.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373796_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373796_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373796_2.csv',\
    '/6 Feb 2012/2373796.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373796.csv','/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373796.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373796.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373796_0.csv']
# 797 = 2-1
files['797'] = ['/25 August 2010/station2-1.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373797_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373797_1.csv','/6 Feb 2012/2373797.csv',\
    '/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373797.csv','/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373797.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373797.csv']
# 798 = 1-1
files['798'] = ['/25 August 2010/station1-1.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373798.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373798_0.csv','/6 Feb 2012/2373798.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373798.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373798_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373798.csv']
# 799 = S-1
files['799'] = ['/25 August 2010/station3-1.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373799_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373799_1.csv','/6 Feb 2012/2373799.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373799.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373799.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373799.csv']
# 800 = 1-4
files['800'] = ['/25 August 2010/station1-4.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373800.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373800_0.csv','/6 Feb 2012/2373800.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373800.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373800_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373800.csv']
# 803 = S-3
files['803'] = ['/25 August 2010/station3-3.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373803_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373803_1.csv','/6 Feb 2012/2373803.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373803.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373803.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373803.csv']
# 804 = 2-3
files['804'] = ['/25 August 2010/station2-3.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373804_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373804_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373804_2.csv',\
    '/6 Feb 2012/2373804.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373804.csv',\
    '/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373804.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373804.csv']
# 807 = 1-2
files['807'] = ['/25 August 2010/station1-2.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373807.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373807_0.csv','/6 Feb 2012/2373807.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373807.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373807_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373807.csv']
# 808 = S-2
files['808'] = ['/25 August 2010/station3-2.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373808_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373808_1.csv','/6 Feb 2012/2373808.csv',\
    '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373808.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373808.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373808.csv']
# 809 = S-4
files['809'] = ['/6 May 2010/Station3-4.csv','/25 August 2010/station3-4.csv',\
    '/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373809_0.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373809_1.csv',\
    '/6 Feb 2012/2373809.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373809.csv',\
    '/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373809.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373809.csv']
# 810 = 2-2
files['810'] = ['/25 August 2010/station2-2.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373810_0.csv',\
    '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373810_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373810_2.csv',\
    '/6 Feb 2012/2373810.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373810.csv',\
    '/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373810.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373810.csv']
# 40 = ILean 30m
files['40'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042540_0_mod.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042540.csv']
# 52 = ILean 25m
files['52'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042552_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042552.csv']
# 44 = ILean 20m
files['44'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042544_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042544.csv']
# 45 = ILean 15m
files['45'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042545_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042545.csv']
# 43 = ILean 10m
files['43'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042543_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042543.csv']
# 36 = ILean 5m
files['36'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042536_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042536.csv']
# 47 = FT 28m
files['47'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042547.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042547.csv']
# 56 = FT 22m
files['56'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042556.csv',\
    '/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042556_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042556.csv']
# 35 = FT 17m
files['35'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042535.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042535.csv']
# 41 = FT 12m
files['41'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042541.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042541.csv']
# 33 = FT 7m
files['33'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042533.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042533.csv']
# 49 = FT 2m
files['49'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042549.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042549.csv']
# 62 = Ursula 25 m
files['62'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042562.csv']
# 58 = Ursula 20 m
files['58'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042558.csv']
# 54 = Ursula 15 m
files['54'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042554.csv']
# 61 = Ursula 10 m
files['61'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042561.csv']
# 42 = Ursula 5 m
files['42'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042542.csv']
# 59 = SMM 17.5 m
files['59'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042559.csv']
# 32 = SMM 15 m
files['32'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042532.csv']
# 60 = SMM 10 m
files['60'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042560.csv']
# 48 = SMM 5 m
files['48'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042548.csv']
# 34 = SMM 2.5 m
files['34'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042534.csv']
# 63 = SMU 17.5 m
files['63'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042563.csv']
# 37 = SMU 15 m
files['37'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042537.csv']
# 66 = SMU 10 m
files['66'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042566.csv']
# 65 = SMU 7.5 m
files['65'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042565.csv']
# 64 = SMU 5 m
files['64'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042564.csv']
# 50 = Freddie 30 m
files['50'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042550.csv']
# 51 = Freddie 24 m
files['51'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042551.csv']
# 53 = Freddie 18 m
files['53'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042553.csv']
# 39 = Freddie 12 m
files['39'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042539.csv']
# 46 = Freddie 6 m
files['46'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042546.csv']
# 38 = road between well 6 and well 5
files['38'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042538.csv']
# 57 = near well 15, top of ridge
files['57'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042557.csv']


In [5]:
# Singling out the late sensors: list of sensors placed in the field on May 7 2012
latesensors = ['62','58','54','61','42','59','32','60','48','34','63','37','66','65','64','50','51','53','39','46']


In [6]:
#Preparing for a loop over all sensors; make a list of all sensors
sensors = files.keys()
sorted(sensors)

['32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '795',
 '796',
 '797',
 '798',
 '799',
 '800',
 '803',
 '804',
 '807',
 '808',
 '809',
 '810']

In [8]:
#preparing for a loop over all sensors
#make some empty dictionary objects for datetime (dt), Temperature (T), and Relative Humidity (RH) data
dt = dict()
T = dict()
RH = dict()

In [9]:
#start the loop
for ss in sensors:
    print(ss) #to show which one we are on
    
    dttmp = np.array([]) #make empty temporary arrays for each variable
    Ttmp = np.array([])
    RHtmp = np.array([])
    
     # loop through files and (1) make record arrays out of their data, (2) delete unecessary rows (3) adjust time
        #if necessary, (4) calibrate data 
    for ff in files[ss]:
        # read file
        data = csv2rec('/Users/Eren/Desktop/data/HOBO/Data'+ff,skiprows=1,comments='!') #This needs to be changed to
        #reflect the local storage path to run on another computer.
        fields = data.dtype.names
        mask = (data.end_of_file=='')#This gets rid of the end of file row by only selecting rows where the end of file
        #field is empty.  
        if 'coupler_detached' in fields:
            mask = mask & (data.coupler_detached=='') #This gets rid of the row that spawned when the coupler detached 
            #during a logging of the data. 
        if 'coupler_attached' in fields:
            mask = mask & (data.coupler_attached=='')#This gets rid of the row that spawned when the coupler attached
            #during a logging of the data. 
        if 'stopped' in fields:
            mask = mask & (data.stopped=='') #This gets rid of the row where the data collection stopped
        Ttmp = np.append(Ttmp,data['temp_c'][mask])
        RHtmp = np.append(RHtmp,data.rh_[mask])
        
        # adjust time
        if len(ss)==3:
            # Kevin's sensors are on daylight savings time - subtract an hour
            dttmp = np.append(dttmp,(data.date_time_gmt0700[mask]-datetime.timedelta(seconds=3600))) 
        elif len(ss)==2:
            dttmp = np.append(dttmp,data.date_time_gmt0800[mask])
            
    # exclude non-field data for sensors placed in field on May 7 2012
    if ss in latesensors:
        mask = dttmp>=datetime.datetime(2012,5,7,17,0,0)
        dttmp = dttmp[mask]
        Ttmp = Ttmp[mask]
        RHtmp = RHtmp[mask]
    # calibrate
    if len(ss)==3: #three-digit sensors use calK calibrations
        mt = calK.mt[calK.sensor==ss]
        bt = calK.bt[calK.sensor==ss]
        mrh = calK.mrh[calK.sensor==ss]
        brh = calK.brh[calK.sensor==ss]
    elif len(ss)==2: #two-digit sensors use calnew calibrations
        mt = calnew.mt[calnew.sensor==ss]
        bt = calnew.bt[calnew.sensor==ss]
        mrh = calnew.mrh[calnew.sensor==ss]
        brh = calnew.brh[calnew.sensor==ss]
    
    Ttmp = mt*Ttmp+bt
    RHtmp = mrh*RHtmp+brh

    # adjust rel hum
    RHtmp[RHtmp>100.] = 100. #Make is so it can't go over 100%

    if ss=='803':
        RHtmp = RHtmp+np.nan  # replace bad RH sensor data with nans
        
    # save date, temp, rel hum
    dt[ss] = dttmp
    T[ss] = Ttmp
    RH[ss] = RHtmp
    
#Create a dictionary object to store all the data
data = dict() 
data['dt'] = dt
data['T'] = T
data['RH'] = RH
pickle.dump(data, open('HOBOdata.p','wb'))

45


/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipyker

40
804


/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:44: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:44: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:46: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipyker

61
798
803
59
64
52
53
809
56
37
43
796
47
795
63
800
60
41
66
58
51
32
808
39
38
49
57
33
50
46
65
44
799
36
810
62
34
807
48
35
42
54
797


In [10]:
#Now, all the data for each sensor from all the different data downloads are in one dictionary array called 'data'. 
#The dictionary groups first by variable (T, RH, datetime), and then by sensor number.
#So for example if we want to look at all of the RH data, we type:
data["RH"]

{'32': array([ 74.92086685,  74.93904857,  74.92793752, ...,  34.6241119 ,
         35.83622666,  36.28470911]),
 '33': array([ 53.64219237,  53.91794848,  54.15633104, ...,  82.26426206,
         78.69458411,  74.25723402]),
 '34': array([ 30.85443502,  30.91908114,  31.72816774, ...,  20.02014933,
         20.31004677,  19.68681777]),
 '35': array([ 54.32097663,  53.83310044,  54.29269395, ...,  70.32594184,
         63.82395629,  58.5330754 ]),
 '36': array([ 98.97225383,  99.08134416,  99.01366775, ...,  53.22502288,
         47.87757663,  48.59373443]),
 '37': array([ 30.81807158,  31.01706042,  31.24332184, ...,  19.93025082,
         20.1413608 ,  20.96357864]),
 '38': array([ 24.98779962,  23.14336501,  24.47972152, ...,  24.67871036,
         27.08677833,  24.58477147]),
 '39': array([ 43.56143801,  42.71396778,  42.42811072, ...,  36.79177712,
         35.1554222 ,  37.03318997]),
 '40': array([ 98.48336754,  97.74801793,  98.59750835, ...,  48.7846425 ,
         43.47861017,

In [11]:
#And if we want to look at a particular sensor's RH data, let's say sensor number 32 we type:
data["RH"]['32']

array([ 74.92086685,  74.93904857,  74.92793752, ...,  34.6241119 ,
        35.83622666,  36.28470911])

In [12]:
#And if we want the datetime and T data for sensor 32 as well, we can use a for loop running over the three dictionary
#keys of T, RH, and dt
keys = data.keys()
for key in keys:
    print(data[key]['32'])


[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 7, 22, 12, 45)
 datetime.datetime(2012, 7, 22, 13, 0)
 datetime.datetime(2012, 7, 22, 13, 15)]
[ 74.92086685  74.93904857  74.92793752 ...,  34.6241119   35.83622666
  36.28470911]
[ 23.88246265  23.97499832  24.067534   ...,  30.26360842  30.38476338
  30.06995129]


In [13]:
#if we want to know how many observations a particular sensor has made, we can type:
len(data["RH"]['49'])

21553

In [14]:
#Because some were installed before the others, some sensors should have made more observations than others. 
#For example, the triple-digit sensors were out in the field the longest:
len(data["RH"]['808'])

46367

In [15]:
#And the late sensors were out in the field the shortest. Sensor 32 is a late sensor:
len(data["RH"]['32'])

7282

In [16]:
#Lets say I want to compare temperature data from a particular day on two opposite sides of the hill. I choose
#Freddie and Flat Top to be my sensor strings. First I want to choose a date where both strings of sensors were 
#deployed. First lets find out when Flat Top was deployed by checking the date range of the bottom sensor:
data["dt"]['49']
#Sensor 49, positioned at 2m off the ground, looks like it was deployed from Feb 3 2012 to Sep 15 2012. 

array([datetime.datetime(2012, 2, 3, 21, 0),
       datetime.datetime(2012, 2, 3, 21, 15),
       datetime.datetime(2012, 2, 3, 21, 30), ...,
       datetime.datetime(2012, 9, 15, 8, 45),
       datetime.datetime(2012, 9, 15, 9, 0),
       datetime.datetime(2012, 9, 15, 9, 15)], dtype=object)

In [17]:
#Lets double check Flat Top's other sensors:
FTsensors = ['49','33','41','35','56','47']
for sensor in FTsensors:
    print('sensor #',sensor)
    print(data["dt"][sensor])
# The exact time of deployment start ranges from 20:45 to 21:15 but they all agree they were installed on Feb 3 2012
#Similarly end of deployment time ranges from 9:15 to 9:30 but they all agree they were taken down on Sep 15 2012. 

sensor # 49
[datetime.datetime(2012, 2, 3, 21, 0) datetime.datetime(2012, 2, 3, 21, 15)
 datetime.datetime(2012, 2, 3, 21, 30) ...,
 datetime.datetime(2012, 9, 15, 8, 45) datetime.datetime(2012, 9, 15, 9, 0)
 datetime.datetime(2012, 9, 15, 9, 15)]
sensor # 33
[datetime.datetime(2012, 2, 3, 21, 15)
 datetime.datetime(2012, 2, 3, 21, 30)
 datetime.datetime(2012, 2, 3, 21, 45) ...,
 datetime.datetime(2012, 9, 15, 8, 45) datetime.datetime(2012, 9, 15, 9, 0)
 datetime.datetime(2012, 9, 15, 9, 15)]
sensor # 41
[datetime.datetime(2012, 2, 3, 21, 0) datetime.datetime(2012, 2, 3, 21, 15)
 datetime.datetime(2012, 2, 3, 21, 30) ...,
 datetime.datetime(2012, 9, 15, 8, 45) datetime.datetime(2012, 9, 15, 9, 0)
 datetime.datetime(2012, 9, 15, 9, 15)]
sensor # 35
[datetime.datetime(2012, 2, 3, 20, 45) datetime.datetime(2012, 2, 3, 21, 0)
 datetime.datetime(2012, 2, 3, 21, 15) ...,
 datetime.datetime(2012, 9, 15, 8, 45) datetime.datetime(2012, 9, 15, 9, 0)
 datetime.datetime(2012, 9, 15, 9, 15)]
sensor

In [18]:
#Now let's look at Freddie's sensors:
FRsensors = ['46','39','53','51','50']
for sensor in FRsensors:
    print('sensor #',sensor)
    print(data["dt"][sensor])
#Freddie's sensors appear to all be 'late sensors', arriving in the field only on May 7 2012 at 17:00 (no range because masked in the for loop).
#Similarly the end of deployment time was earlier, on July 22 at 13:00. So if we want to compare Flat Top to Freddie,
#We are limited to this May 7-July 22 window in 2012. 

sensor # 46
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 7, 22, 12, 30)
 datetime.datetime(2012, 7, 22, 12, 45)
 datetime.datetime(2012, 7, 22, 13, 0)]
sensor # 39
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 7, 22, 12, 30)
 datetime.datetime(2012, 7, 22, 12, 45)
 datetime.datetime(2012, 7, 22, 13, 0)]
sensor # 53
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 7, 22, 12, 30)
 datetime.datetime(2012, 7, 22, 12, 45)
 datetime.datetime(2012, 7, 22, 13, 0)]
sensor # 51
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 9, 15, 12, 30)
 datetime.datetime(2012, 9, 15, 12, 45)
 datetime.datetime(2012, 9, 15, 

In [19]:
#To compare, lets make a plot that contains all the data from Freddie's sensors and Flat Top's sensors for one day
#on May 7th. 
plt.figure
plt.plot_date(data["dt"]['50'][0:112], data["T"]['50'][0:112], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][0:112], data["T"]['51'][0:112], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][0:112], data["T"]['53'][0:112], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][0:112], data["T"]['39'][0:112], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][0:112], data["T"]['46'][0:112], '-', markersize=3, markeredgewidth=0, label='FR6'),
#Note: it took a while to figure out which indexes to use to get the same date range for these. There must be a 
#better way; I need to get someone to teach me. Also there is something wrong with the data for sensor 56 around the
#9006th entry. The datetime inverts for about 10ish entries. 
plt.plot_date(data["dt"]['49'][9007:9119], data["T"]['49'][9007:9119], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][9006:9118], data["T"]['33'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][9006:9118], data["T"]['41'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][9006:9118], data["T"]['35'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][9024:9136], data["T"]['56'][9024:9136], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][9006:9118], data["T"]['47'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature, May 7 2012 17:00 to May 8 2012 18:00')
plt.legend(bbox_to_anchor=(.55, 1))
plt.show()

#This plot shows a lot of between slope difference!


In [21]:
#What does it look like on the scale of a week?
plt.figure
plt.plot_date(data["dt"]['50'][0:712], data["T"]['50'][0:712], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][0:712], data["T"]['51'][0:712], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][0:712], data["T"]['53'][0:712], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][0:712], data["T"]['39'][0:712], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][0:712], data["T"]['46'][0:712], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['49'][9007:9719], data["T"]['49'][9007:9719], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][9006:9718], data["T"]['33'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][9006:9718], data["T"]['41'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][9006:9718], data["T"]['35'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][9024:9736], data["T"]['56'][9024:9736], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][9006:9718], data["T"]['47'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

#It looks like the huge between-slope difference is damped after about 3 days. Cloudy?

In [22]:
#Let's look at the next week.
plt.figure
plt.plot_date(data["dt"]['50'][712:1424], data["T"]['50'][712:1424], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][712:1424], data["T"]['51'][712:1424], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][712:1424], data["T"]['53'][712:1424], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][712:1424], data["T"]['39'][712:1424], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][712:1424], data["T"]['46'][712:1424], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['49'][9719:10431], data["T"]['49'][9719:10431], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][9718:10430], data["T"]['33'][9718:10430], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][9718:10430], data["T"]['41'][9718:10430], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][9718:10430], data["T"]['35'][9718:10430], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][9736:10448], data["T"]['56'][9736:10448], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][9718:10430], data["T"]['47'][9718:10430], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

#Differences between slopes are still damped. So maybe the initial data was abberant?

In [23]:
#Looking at the next week: 
plt.figure
plt.plot_date(data["dt"]['50'][1424:2136], data["T"]['50'][1424:2136], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][1424:2136], data["T"]['51'][1424:2136], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][1424:2136], data["T"]['53'][1424:2136], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][1424:2136], data["T"]['39'][1424:2136], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][1424:2136], data["T"]['46'][1424:2136], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['49'][10431:11143], data["T"]['49'][10431:11143], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][10430:11142], data["T"]['33'][10430:11142], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][10430:11142], data["T"]['41'][10430:11142], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][10430:11142], data["T"]['35'][10430:11142], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][10448:11160], data["T"]['56'][10448:11160], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][10430:11142], data["T"]['47'][10430:11142], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

In [24]:
#Looking at the next week: 
plt.figure
plt.plot_date(data["dt"]['50'][2136:2848], data["T"]['50'][2136:2848], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][2136:2848], data["T"]['51'][2136:2848], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][2136:2848], data["T"]['53'][2136:2848], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][2136:2848], data["T"]['39'][2136:2848], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][2136:2848], data["T"]['46'][2136:2848], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['49'][11143:11855], data["T"]['49'][11143:11855], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][11142:11854], data["T"]['33'][11142:11854], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][11142:11854], data["T"]['41'][11142:11854], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][11142:11854], data["T"]['35'][11142:11854], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][11160:11872], data["T"]['56'][11160:11872], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][11142:11854], data["T"]['47'][11142:11854], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

In [25]:
#Let's do the same for another day, this time in July. 
plt.figure
# plt.plot_date(data["dt"]['50'][6650:6762], data["T"]['50'][6650:6762], '-', markersize=3, markeredgewidth=0, label='FR30'),
# plt.plot_date(data["dt"]['51'][6650:6762], data["T"]['51'][6650:6762], '-', markersize=3, markeredgewidth=0, label='FR24'),
# plt.plot_date(data["dt"]['53'][6650:6762], data["T"]['53'][6650:6762], '-', markersize=3, markeredgewidth=0, label='FR18'),
# plt.plot_date(data["dt"]['39'][6650:6762], data["T"]['39'][6650:6762], '-', markersize=3, markeredgewidth=0, label='FR12'),
# plt.plot_date(data["dt"]['46'][6650:6762], data["T"]['46'][6650:6762], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['49'][15657:15769], data["T"]['49'][15657:15769], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][15656:15768], data["T"]['33'][15656:15768], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][15656:15768], data["T"]['41'][15656:15768], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][15656:15768], data["T"]['35'][15656:15768], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][15674:15786], data["T"]['56'][15674:15786], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][15656:15768], data["T"]['47'][15656:15768], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie vs Flat Top Hobo sensors Temperature, July 15 2012 23:30 to July 16 2012 14:00')
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

#Generally I can't find another time that shows that level of separation between slopes as seen in early May. 
#Maybe the radiation shields weren't put on properly at first? 
#Maybe the sensors themselves degrade data quality quickly? Interesting how the temperature peaks are never smooth
#again, either. So is this a combination of added noise and reduced sensitivity? 
#Was there some kind of time-offset in the data in early May? This is not reflected in the datetime; any chance the
#datetimes are wrong?
#Could the early May feature actually be realistic conditions? a 10 degree difference between slopes? (I think so).

In [26]:
#We've seen that generally there isn't much T difference between Freddie and Flat Top
#Let's now compare upslope to downslope. Flat Top to Ursula first. What's the date range on Ursula's sensors?
URsensors = ['62','58','54','61','42']
for sensor in URsensors:
    print('sensor #',sensor)
    print(data["dt"][sensor])
#These all also appear to be late sensors. So I will use the same indices in these charts as for Freddie's sensors.

sensor # 62
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 9, 15, 10, 0)
 datetime.datetime(2012, 9, 15, 10, 15)
 datetime.datetime(2012, 9, 15, 10, 30)]
sensor # 58
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 9, 15, 10, 15)
 datetime.datetime(2012, 9, 15, 10, 30)
 datetime.datetime(2012, 9, 15, 10, 45)]
sensor # 54
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 9, 15, 10, 15)
 datetime.datetime(2012, 9, 15, 10, 30)
 datetime.datetime(2012, 9, 15, 10, 45)]
sensor # 61
[datetime.datetime(2012, 5, 7, 17, 0) datetime.datetime(2012, 5, 7, 17, 15)
 datetime.datetime(2012, 5, 7, 17, 30) ...,
 datetime.datetime(2012, 9, 15, 10, 0)
 datetime.datetime(2012, 9, 15, 10, 15)
 datetime.datetime(2012, 9, 15,

In [89]:
#We've seen that generally there isn't much T difference between Freddie and Flat Top
#Let's now compare upslope to downslope. Flat Top to Ursula first. 
plt.figure
plt.plot_date(data["dt"]['62'][0:112], data["T"]['62'][0:112], '-', markersize=3, markeredgewidth=0, label='UR25'),
plt.plot_date(data["dt"]['58'][0:112], data["T"]['58'][0:112], '-', markersize=3, markeredgewidth=0, label='UR20'),
plt.plot_date(data["dt"]['54'][0:112], data["T"]['54'][0:112], '-', markersize=3, markeredgewidth=0, label='UR15'),
plt.plot_date(data["dt"]['61'][0:112], data["T"]['61'][0:112], '-', markersize=3, markeredgewidth=0, label='UR10'),
plt.plot_date(data["dt"]['42'][0:112], data["T"]['42'][0:112], '-', markersize=3, markeredgewidth=0, label='UR5'),

plt.plot_date(data["dt"]['49'][9007:9119], data["T"]['49'][9007:9119], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][9006:9118], data["T"]['33'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][9006:9118], data["T"]['41'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][9006:9118], data["T"]['35'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][9024:9136], data["T"]['56'][9024:9136], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][9006:9118], data["T"]['47'][9006:9118], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Ursula vs Flat Top Hobo sensors Temperature, May 7 2012 17:00 to May 8 2012 18:00')
plt.legend(bbox_to_anchor=(.55, 1))
plt.show()
#Ursula also has the smooth high amplitude data at the start.

In [91]:
#What does it look like on the scale of a week?
plt.figure
plt.plot_date(data["dt"]['62'][0:712], data["T"]['62'][0:712], '-', markersize=3, markeredgewidth=0, label='UR25'),
plt.plot_date(data["dt"]['58'][0:712], data["T"]['58'][0:712], '-', markersize=3, markeredgewidth=0, label='UR20'),
plt.plot_date(data["dt"]['54'][0:712], data["T"]['54'][0:712], '-', markersize=3, markeredgewidth=0, label='UR15'),
plt.plot_date(data["dt"]['61'][0:712], data["T"]['61'][0:712], '-', markersize=3, markeredgewidth=0, label='UR10'),
plt.plot_date(data["dt"]['42'][0:712], data["T"]['42'][0:712], '-', markersize=3, markeredgewidth=0, label='UR5'),

plt.plot_date(data["dt"]['49'][9007:9719], data["T"]['49'][9007:9719], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][9006:9718], data["T"]['33'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][9006:9718], data["T"]['41'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][9006:9718], data["T"]['35'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][9024:9736], data["T"]['56'][9024:9736], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][9006:9718], data["T"]['47'][9006:9718], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Ursula vs Flat Top Hobo sensors Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()


In [ ]:
#suspicious that Freddie and Ursula look so similar when FlatTop looks different. Compare Freddie and Ursula
#Freddie vs. Ursula May 7
plt.figure
plt.plot_date(data["dt"]['62'][0:112], data["T"]['62'][0:112], '-', markersize=3, markeredgewidth=0, label='UR25'),
plt.plot_date(data["dt"]['58'][0:112], data["T"]['58'][0:112], '-', markersize=3, markeredgewidth=0, label='UR20'),
plt.plot_date(data["dt"]['54'][0:112], data["T"]['54'][0:112], '-', markersize=3, markeredgewidth=0, label='UR15'),
plt.plot_date(data["dt"]['61'][0:112], data["T"]['61'][0:112], '-', markersize=3, markeredgewidth=0, label='UR10'),
plt.plot_date(data["dt"]['42'][0:112], data["T"]['42'][0:112], '-', markersize=3, markeredgewidth=0, label='UR5'),
plt.plot_date(data["dt"]['50'][0:112], data["T"]['50'][0:112], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][0:112], data["T"]['51'][0:112], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][0:112], data["T"]['53'][0:112], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][0:112], data["T"]['39'][0:112], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][0:112], data["T"]['46'][0:112], '-', markersize=3, markeredgewidth=0, label='FR6'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Ursula vs Freddie Hobo sensors Temperature, May 7 2012 17:00 to May 8 2012 18:00')
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

#appears to show no difference

In [101]:
#Question: is this smooth high-amplitude data some kind of abberation at the start of deployment?
#Flat Top start data:
plt.figure
plt.plot_date(data["dt"]['49'][400:1100], data["T"]['49'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT2'),
plt.plot_date(data["dt"]['33'][400:1100], data["T"]['33'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT7'),
plt.plot_date(data["dt"]['41'][400:1100], data["T"]['41'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT12'),
plt.plot_date(data["dt"]['35'][400:1100], data["T"]['35'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT17'),
plt.plot_date(data["dt"]['56'][400:1100], data["T"]['56'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT22'),
plt.plot_date(data["dt"]['47'][400:1100], data["T"]['47'][400:1100], '-', markersize=3, markeredgewidth=0, label='FT28'),
plt.xticks(rotation=70), plt.grid(True), plt.title('Flat Top Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()
#It's not exactly the same problem, but the start data sure looks whacky here as well. Perhaps that supports the idea
#that the first week or so of data should just be ignored? Yet this is slightly troublesome to assume because Percy's
#code is already masking for the relevant dates and chopping off bad data. But perhaps she never delved into this
#data too deeply and thus didn't notice the weird stuff?
#Note, I chopped off the first 400 data points after reading the read-me wihch says they weren't deployed until Feb 7,
#even though the data stream starts Feb 3. After Feb 7 the data does not look smooth or high amplitude, so the May 7
#data still seems weird. The Pre-deployment data for this data stream look more the way I would expect pre-deployment
#data to look-- lower amplitude rather than higher. 
#Note, interesting that there's evidence of a temperature inversion midday Feb 8 and 9. Even up to 28m. 

In [103]:
#Let's start looking at SMU
plt.figure
plt.plot_date(data["dt"]['63'][0:112], data["T"]['63'][0:112], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][0:112], data["T"]['37'][0:112], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][0:112], data["T"]['66'][0:112], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][0:112], data["T"]['65'][0:112], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][0:112], data["T"]['64'][0:112], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.xticks(rotation=70), plt.grid(True), plt.title('SMU May 7 2012')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

#Looks like some sensors go offline. Is this still true on the scale of a week?

In [104]:
plt.figure
plt.plot_date(data["dt"]['63'][0:712], data["T"]['63'][0:712], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][0:712], data["T"]['37'][0:712], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][0:712], data["T"]['66'][0:712], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][0:712], data["T"]['65'][0:712], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][0:712], data["T"]['64'][0:712], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.xticks(rotation=70), plt.grid(True), plt.title('SMU Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()
#What is going on?

In [105]:
#look closer
plt.figure
plt.plot_date(data["dt"]['63'][100:512], data["T"]['63'][100:512], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][100:512], data["T"]['37'][100:512], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][100:512], data["T"]['66'][100:512], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][100:512], data["T"]['65'][100:512], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][100:512], data["T"]['64'][100:512], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.xticks(rotation=70), plt.grid(True), plt.title('SMU Temperature')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()
#okay it appears that Sensor 64 doesn't actually begin making data until May 9 at 12:45. So start all other SMU sensors
#at index 175, start 64 at 0. 


In [119]:
#Looking at SMU with the correct start times:
plt.figure
plt.plot_date(data["dt"]['63'][175:412], data["T"]['63'][175:412], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][175:412], data["T"]['37'][175:412], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][175:412], data["T"]['66'][175:412], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][175:412], data["T"]['65'][175:412], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][0:237], data["T"]['64'][0:237], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.xticks(rotation=70), plt.grid(True), plt.title('SMU May 9 2012')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

#Sensor 64 still seems to be off. 

In [128]:
#Let's start looking May 11th and onward to ignore any weird data things, and lets zoom in on three days:
plt.figure
plt.plot_date(data["dt"]['63'][316:616], data["T"]['63'][316:616], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][316:616], data["T"]['37'][316:616], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][316:616], data["T"]['66'][316:616], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][316:616], data["T"]['65'][316:616], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][141:341], data["T"]['64'][141:341], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.xticks(rotation=70), plt.grid(True), plt.title('SMU May 11-13 2012')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

In [143]:
#Let's compare SMU to Flat Top during this range. I've thrown Ursula and Freddie on there as well so we can compare 
#by commenting/uncommenting

plt.figure
# plt.plot_date(data["dt"]['62'][316:712], data["T"]['62'][316:712], '-', markersize=3, markeredgewidth=0, label='UR25'),
# plt.plot_date(data["dt"]['58'][316:712], data["T"]['58'][316:712], '-', markersize=3, markeredgewidth=0, label='UR20'),
# plt.plot_date(data["dt"]['54'][316:712], data["T"]['54'][316:712], '-', markersize=3, markeredgewidth=0, label='UR15'),
# plt.plot_date(data["dt"]['61'][316:712], data["T"]['61'][316:712], '-', markersize=3, markeredgewidth=0, label='UR10'),
# plt.plot_date(data["dt"]['42'][316:712], data["T"]['42'][316:712], '-', markersize=3, markeredgewidth=0, label='UR5'),

plt.plot_date(data["dt"]['50'][316:712], data["T"]['50'][316:712], '-', markersize=3, markeredgewidth=0, label='FR30'),
plt.plot_date(data["dt"]['51'][316:712], data["T"]['51'][316:712], '-', markersize=3, markeredgewidth=0, label='FR24'),
plt.plot_date(data["dt"]['53'][316:712], data["T"]['53'][316:712], '-', markersize=3, markeredgewidth=0, label='FR18'),
plt.plot_date(data["dt"]['39'][316:712], data["T"]['39'][316:712], '-', markersize=3, markeredgewidth=0, label='FR12'),
plt.plot_date(data["dt"]['46'][316:712], data["T"]['46'][316:712], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['63'][316:712], data["T"]['63'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
plt.plot_date(data["dt"]['37'][316:712], data["T"]['37'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU15'),
plt.plot_date(data["dt"]['66'][316:712], data["T"]['66'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU10'),
plt.plot_date(data["dt"]['65'][316:712], data["T"]['65'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
plt.plot_date(data["dt"]['64'][141:437], data["T"]['64'][141:437], '-', markersize=3, markeredgewidth=0, label='SMU5'),

# plt.plot_date(data["dt"]['49'][9323:9719], data["T"]['49'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT2'),
# plt.plot_date(data["dt"]['33'][9322:9718], data["T"]['33'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT7'),
# plt.plot_date(data["dt"]['41'][9322:9718], data["T"]['41'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT12'),
# plt.plot_date(data["dt"]['35'][9323:9719], data["T"]['35'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT17'),
# plt.plot_date(data["dt"]['56'][9340:9736], data["T"]['56'][9340:9736], '-', markersize=3, markeredgewidth=0, label='FT22'),
# plt.plot_date(data["dt"]['47'][9322:9718], data["T"]['47'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT28'),


plt.xticks(rotation=70), plt.grid(True), plt.title('SMU vs Freddie Hobo Temperature May 11-13')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()



In [151]:
#Let's make a plot of each sensor string's variance between top and bottom sensor.
plt.figure
plt.plot_date(data["dt"]['62'][316:712], data["T"]['62'][316:712]-data["T"]['42'][316:712], '-', markersize=3, markeredgewidth=0, label='UR25-UR5'),
# plt.plot_date(data["dt"]['58'][316:712], data["T"]['58'][316:712], '-', markersize=3, markeredgewidth=0, label='UR20'),
# plt.plot_date(data["dt"]['54'][316:712], data["T"]['54'][316:712], '-', markersize=3, markeredgewidth=0, label='UR15'),
# plt.plot_date(data["dt"]['61'][316:712], data["T"]['61'][316:712], '-', markersize=3, markeredgewidth=0, label='UR10'),
#plt.plot_date(data["dt"]['42'][316:712], data["T"]['42'][316:712], '-', markersize=3, markeredgewidth=0, label='UR5'),

plt.plot_date(data["dt"]['50'][316:712], data["T"]['50'][316:712]-data["T"]['46'][316:712], '-', markersize=3, markeredgewidth=0, label='FR30-FR6')
# plt.plot_date(data["dt"]['51'][316:712], data["T"]['51'][316:712], '-', markersize=3, markeredgewidth=0, label='FR24'),
# plt.plot_date(data["dt"]['53'][316:712], data["T"]['53'][316:712], '-', markersize=3, markeredgewidth=0, label='FR18'),
# plt.plot_date(data["dt"]['39'][316:712], data["T"]['39'][316:712], '-', markersize=3, markeredgewidth=0, label='FR12'),
# plt.plot_date(data["dt"]['46'][316:712], data["T"]['46'][316:712], '-', markersize=3, markeredgewidth=0, label='FR6'),

plt.plot_date(data["dt"]['63'][316:712], data["T"]['63'][316:712]-data["T"]['65'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU17.5-SMU7.5'),
# plt.plot_date(data["dt"]['37'][316:712], data["T"]['37'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU15'),
# plt.plot_date(data["dt"]['66'][316:712], data["T"]['66'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU10'),
# plt.plot_date(data["dt"]['65'][316:712], data["T"]['65'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
# plt.plot_date(data["dt"]['64'][141:437], data["T"]['64'][141:437], '-', markersize=3, markeredgewidth=0, label='SMU5'),

plt.plot_date(data["dt"]['49'][9323:9719], data["T"]['47'][9322:9718]-data["T"]['49'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT28-FT2'),
# plt.plot_date(data["dt"]['33'][9322:9718], data["T"]['33'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT7'),
# plt.plot_date(data["dt"]['41'][9322:9718], data["T"]['41'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT12'),
# plt.plot_date(data["dt"]['35'][9323:9719], data["T"]['35'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT17'),
# plt.plot_date(data["dt"]['56'][9340:9736], data["T"]['56'][9340:9736], '-', markersize=3, markeredgewidth=0, label='FT22'),
# plt.plot_date(data["dt"]['47'][9322:9718], data["T"]['47'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT28'),


plt.xticks(rotation=70), plt.grid(True), plt.title('Top-Bottom Temperature for 4 strings May 11-13')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

#This is interesting! Notice how during the middle of the day, the top is cooler than the bottom for May 11 and 12.
#Effect of transpiration?

In [29]:
#Let's make a plot of between slope difference at each level.
plt.figure
# plt.plot_date(data["dt"]['62'][316:712], data["T"]['62'][316:712], '-', markersize=3, markeredgewidth=0, label='UR25'),
# plt.plot_date(data["dt"]['58'][316:712], data["T"]['58'][316:712], '-', markersize=3, markeredgewidth=0, label='UR20'),
# plt.plot_date(data["dt"]['54'][316:712], data["T"]['54'][316:712], '-', markersize=3, markeredgewidth=0, label='UR15'),
# plt.plot_date(data["dt"]['61'][316:712], data["T"]['61'][316:712], '-', markersize=3, markeredgewidth=0, label='UR10'),
# plt.plot_date(data["dt"]['42'][316:712], data["T"]['42'][316:712], '-', markersize=3, markeredgewidth=0, label='UR5'),

plt.plot_date(data["dt"]['50'][316:712], data["T"]['50'][316:712]-data["T"]['47'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FR30-FT28'),
plt.plot_date(data["dt"]['51'][316:712], data["T"]['51'][316:712]-data["T"]['56'][9340:9736], '-', markersize=3, markeredgewidth=0, label='FR24-FT22'),
plt.plot_date(data["dt"]['53'][316:712], data["T"]['53'][316:712]-data["T"]['35'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FR18-FT17'),
plt.plot_date(data["dt"]['39'][316:712], data["T"]['39'][316:712]-data["T"]['41'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FR12-FT12'),
plt.plot_date(data["dt"]['46'][316:712], data["T"]['46'][316:712]-data["T"]['49'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FR6-FT2'),

# plt.plot_date(data["dt"]['63'][316:712], data["T"]['63'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU17.5'),
# plt.plot_date(data["dt"]['37'][316:712], data["T"]['37'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU15'),
# plt.plot_date(data["dt"]['66'][316:712], data["T"]['66'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU10'),
# plt.plot_date(data["dt"]['65'][316:712], data["T"]['65'][316:712], '-', markersize=3, markeredgewidth=0, label='SMU7.5'),
# plt.plot_date(data["dt"]['64'][141:437], data["T"]['64'][141:437], '-', markersize=3, markeredgewidth=0, label='SMU5'),

# plt.plot_date(data["dt"]['49'][9323:9719], data["T"]['49'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT2'),
# plt.plot_date(data["dt"]['33'][9322:9718], data["T"]['33'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT7'),
# plt.plot_date(data["dt"]['41'][9322:9718], data["T"]['41'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT12'),
# plt.plot_date(data["dt"]['35'][9323:9719], data["T"]['35'][9323:9719], '-', markersize=3, markeredgewidth=0, label='FT17'),
# plt.plot_date(data["dt"]['56'][9340:9736], data["T"]['56'][9340:9736], '-', markersize=3, markeredgewidth=0, label='FT22'),
# plt.plot_date(data["dt"]['47'][9322:9718], data["T"]['47'][9322:9718], '-', markersize=3, markeredgewidth=0, label='FT28'),

plt.xticks(rotation=70), plt.grid(True), plt.title('Freddie-Flat Top Temperature at each height, May 11-13')
plt.legend(bbox_to_anchor=(1.15, 1))
plt.show()

In [28]:
#Let's make plots showing anomalies, i.e. subtracting out the mean. Let's first do 

data["dt"]['50'][316:712]

array([datetime.datetime(2012, 5, 11, 0, 0),
       datetime.datetime(2012, 5, 11, 0, 15),
       datetime.datetime(2012, 5, 11, 0, 30),
       datetime.datetime(2012, 5, 11, 0, 45),
       datetime.datetime(2012, 5, 11, 1, 0),
       datetime.datetime(2012, 5, 11, 1, 15),
       datetime.datetime(2012, 5, 11, 1, 30),
       datetime.datetime(2012, 5, 11, 1, 45),
       datetime.datetime(2012, 5, 11, 2, 0),
       datetime.datetime(2012, 5, 11, 2, 15),
       datetime.datetime(2012, 5, 11, 2, 30),
       datetime.datetime(2012, 5, 11, 2, 45),
       datetime.datetime(2012, 5, 11, 3, 0),
       datetime.datetime(2012, 5, 11, 3, 15),
       datetime.datetime(2012, 5, 11, 3, 30),
       datetime.datetime(2012, 5, 11, 3, 45),
       datetime.datetime(2012, 5, 11, 4, 0),
       datetime.datetime(2012, 5, 11, 4, 15),
       datetime.datetime(2012, 5, 11, 4, 30),
       datetime.datetime(2012, 5, 11, 4, 45),
       datetime.datetime(2012, 5, 11, 5, 0),
       datetime.datetime(2012, 5, 11, 5,

In [142]:
data["dt"]['56'][9340:9718]

array([datetime.datetime(2012, 5, 11, 0, 0),
       datetime.datetime(2012, 5, 11, 0, 15),
       datetime.datetime(2012, 5, 11, 0, 30),
       datetime.datetime(2012, 5, 11, 0, 45),
       datetime.datetime(2012, 5, 11, 1, 0),
       datetime.datetime(2012, 5, 11, 1, 15),
       datetime.datetime(2012, 5, 11, 1, 30),
       datetime.datetime(2012, 5, 11, 1, 45),
       datetime.datetime(2012, 5, 11, 2, 0),
       datetime.datetime(2012, 5, 11, 2, 15),
       datetime.datetime(2012, 5, 11, 2, 30),
       datetime.datetime(2012, 5, 11, 2, 45),
       datetime.datetime(2012, 5, 11, 3, 0),
       datetime.datetime(2012, 5, 11, 3, 15),
       datetime.datetime(2012, 5, 11, 3, 30),
       datetime.datetime(2012, 5, 11, 3, 45),
       datetime.datetime(2012, 5, 11, 4, 0),
       datetime.datetime(2012, 5, 11, 4, 15),
       datetime.datetime(2012, 5, 11, 4, 30),
       datetime.datetime(2012, 5, 11, 4, 45),
       datetime.datetime(2012, 5, 11, 5, 0),
       datetime.datetime(2012, 5, 11, 5,

In [92]:
data["dt"]['47']

data["dt"]['47'][15656:16068]

array([datetime.datetime(2012, 2, 3, 21, 15),
       datetime.datetime(2012, 2, 3, 21, 30),
       datetime.datetime(2012, 2, 3, 21, 45), ...,
       datetime.datetime(2012, 9, 15, 9, 0),
       datetime.datetime(2012, 9, 15, 9, 15),
       datetime.datetime(2012, 9, 15, 9, 30)], dtype=object)